In [1]:
### Importing dependencies to work with the data and create the model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, log_loss
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd 
import psycopg2 
import pandas.io.sql as sqlio 
from password import db_password



In [2]:
### Create connection with database and load database
connection = psycopg2.connect(user='postgres', password=db_password, 
host='127.0.0.1',
port='5432', 
database='Startup-Analysis')
sql = "SELECT * FROM startup_alldata"
data = sqlio.read_sql_query(sql,connection)

In [3]:
### Check database head to see if the upload was done correct
data.head()

,state_code,year_,vp_gdp,labels,founded_at,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,...,has_angel,has_rounda,has_roundb,has_roundc,has_roundd,avg_participants,is_top500,reached_milestone,founded_first_funding_days_difference,first_last_funding_days_difference
0,CA,2007,1.9,1,2007,2.2493,3.0027,4.6685,6.7041,3,...,1,0,0,0,0,1.0000,0,1,821,275
1,CA,2000,7.7,1,2000,5.1260,9.9973,7.0055,7.0055,9,...,0,0,1,1,1,4.7500,1,1,1871,1778
2,CA,2009,3.2,1,2009,1.0329,1.0329,1.4575,2.2055,5,...,0,1,0,0,0,4.0000,1,1,377,0
3,CA,2002,1.8,1,2002,3.1315,5.3151,6.0027,6.0027,5,...,0,0,1,1,1,3.3333,1,1,1143,797
4,CA,2010,2.0,0,2010,0.0000,1.6685,0.0384,0.0384,2,...,1,0,0,0,0,1.0000,1,1,0,609


In [37]:
### 2 columns contain NA, this data is valuable so changing NA to value 999, the reason is the columns have values 0 by default, taking down the
## the columns we miss almost all 0 labels and the model wouldn´t couldn´t be trained
data["age_first_milestone_year"] = data["age_first_milestone_year"].fillna(value=999)
data["age_last_milestone_year"] = data["age_last_milestone_year"].fillna(value=999)

In [38]:
X = data.drop(["labels", "state_code", "founded_at", "year_"], axis= 1)
y = data["labels"]

Using standard methodology to separate train and test data, after that we are going to scale with StandardScaler from sklearn.preprocessing so we don´t have outliers

In [39]:
### Split the data into test and train 
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [40]:
### Going to scale the data to avoid outliers 
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Applying Gradient Boosting Classifier, Boosting has the idea of learning from each successful model ran and minimize the error. In a summarized way to see it the Gradient Boosting Classifier incorporates a minimize cost function to cast the next model

In [45]:


classifier = GradientBoostingClassifier(n_estimators=200,
   learning_rate=0.05,
   max_features=5,
   max_depth=3,
   random_state=0)

classifier.fit(X_train_scaled, y_train)
print("Accuracy score (training): {0:.2f}".format(
classifier.score(X_train_scaled, y_train)))
print("Accuracy score (validation): {0:.2f}".format(
classifier.score(X_test_scaled, y_test)))
y_pred = classifier.predict(X_test_scaled)



Accuracy score (training): 0.90
Accuracy score (validation): 0.80


In [47]:
print(classification_report(y_pred, y_test))
print(accuracy_score(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.63      0.79      0.70        66
           1       0.90      0.81      0.85       161

    accuracy                           0.80       227
   macro avg       0.76      0.80      0.78       227
weighted avg       0.82      0.80      0.81       227

0.801762114537445


Creating this model we had a 80% acurracy, the model does what we need, with a 79% recall for 0 and 81% we have an almost certainty that we can detect the true successful or failed startups and with this we can forecast for new business

And with the help of R we can detect the variables that statistically matters for our model